<a href="https://colab.research.google.com/github/pizzapuppies/Usif_FYP_code/blob/main/Kfold_%20hyper_tuned_concentration_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Updates: 
- joined all the negtive cases 
- 20% val split is better than 10%
- currently trying smote

In [2]:
! pip install -q -U keras-tuner
from google.colab import drive
import pandas as pd 
import os
import tensorflow as tf
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import csv
import random
from tensorflow import keras
import sys
from matplotlib import pyplot as plt
import keras_tuner as kt
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Data preprocessing

In [3]:
#@title spliting the data

data = {"n":[]}

with open("/content/gdrive/My Drive/data.csv","r") as rfile:
  line = 0

  reader  = csv.reader(rfile)
  real,imag = [],[]

  for row in reader:
    label = row[0] if "n" not in row[0] else "n"
    
    if label not in data.keys():
      data[label] = []
    temp =[]
    real,imag = list(map(float,row[1:51])),list(map(float,row[51:]))
    for i in range(len(real)):
      temp.append([real[i],imag[i]])
    
    
    data[label].append(temp)




lim = 70
for key in data.keys():
  temp = data[key]
  while len(temp) < lim:
    r = random.randint(0,len(data[key])-1)
    temp.append(data[key][r])

  
split = 14 #number of test cases per class
x_train = []
y_train = []
x_test =  []
y_test =  []
X,y = [],[]

for key in data.keys():
  X = X + data[key]
  for _ in range(len(data[key])):
    y.append(key)
  
  
  

# shuffle the test and train 

temp = list(zip(X, y))
random.shuffle(temp)
X,y = zip(*temp)
X,y = list(X), list(y)

X = np.array(X)

In [4]:
mapping = {}
codes = list(data.keys())
l = len(codes)
for i in range(l):
  li = [0]*l
  
  li[i] = 1
  
  mapping[codes[i]] = np.array(li)

temp = []
for i in y:
  temp.append(mapping[i])
y= np.array(temp)



# building and runnning the model

In [5]:
def model_builder(hp):
  model = tf.keras.models.Sequential()
  model.add(keras.layers.Dense(128,         #Number of nodes
                        input_shape=(50,2), #Number of input variables
                        name='Hidden-Layer-1', #Logical name
                        activation='relu'))    #activation function

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  
  #Add a second hidden layer
  model.add(keras.layers.Dense(units=hp_units,
                                name='Hidden-Layer-2',
                                activation='relu',
                                ))

  #Add a second hidden layer
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-3',
                                activation='relu'))

  model.add(tf.keras.layers.Flatten())

  #Add a second hidden layer
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-4',
                              activation='relu'))
  
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-5',
                              activation='relu'))
  
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-6',
                              activation='relu'))


  model.add(keras.layers.Dense(l,
                             name='Output-Layer',
                             activation='softmax'))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  
  

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

def learn_test(X_train,Y_train,X_test,Y_test):
  

  tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  tuner.search(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[stop_early],verbose = 0)

  # Get the optimal hyperparameters
  best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

  #print(f
  """
  The hyperparameter search is complete. The optimal number of units in the first densely-connected
  layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
  is {best_hps.get('learning_rate')}.
  )"""

 # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
  model = tuner.hypermodel.build(best_hps)
  history = model.fit(X_train, Y_train, epochs=100, validation_split=0.2)

  val_acc_per_epoch = history.history['val_accuracy']
  best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
  print('Best epoch: %d' % (best_epoch,))

  hypermodel = tuner.hypermodel.build(best_hps)

  # Retrain the model
  hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

  eval_result = hypermodel.evaluate(X_test, Y_test)
  print("[test loss, test accuracy]:", eval_result)
  return eval_result[1]


# final result

In [6]:
from sklearn.model_selection import KFold

n_split=10

li = []
li1 = 0

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test=X[train_index],X[test_index]
  y_train,y_test=y[train_index],y[test_index]
  xa = learn_test(x_train,y_train,x_test,y_test)
  li.append(round(xa*100))
  li1 +=1
     


Epoch 1/100
13/13 [==============================] - 3s 67ms/step - loss: 111.4852 - accuracy: 0.1117 - val_loss: 12.6684 - val_accuracy: 0.2475
Epoch 2/100
13/13 [==============================] - 0s 34ms/step - loss: 5.8342 - accuracy: 0.1886 - val_loss: 2.1018 - val_accuracy: 0.3168
Epoch 3/100
13/13 [==============================] - 0s 36ms/step - loss: 2.1701 - accuracy: 0.2829 - val_loss: 1.7747 - val_accuracy: 0.3465
Epoch 4/100
13/13 [==============================] - 1s 41ms/step - loss: 1.6671 - accuracy: 0.3052 - val_loss: 1.5241 - val_accuracy: 0.4158
Epoch 5/100
13/13 [==============================] - 0s 33ms/step - loss: 1.4813 - accuracy: 0.4119 - val_loss: 1.3055 - val_accuracy: 0.4653
Epoch 6/100
13/13 [==============================] - 0s 33ms/step - loss: 1.3076 - accuracy: 0.4839 - val_loss: 1.1368 - val_accuracy: 0.5149
Epoch 7/100
13/13 [==============================] - 0s 35ms/step - loss: 1.2534 - accuracy: 0.4938 - val_loss: 1.2473 - val_accuracy: 0.4158
Epo

In [8]:
print("The highest accuracy acheived is ", max(li))


[96.42857313156128, 98.21428656578064, 98.21428656578064, 55.35714030265808, 98.21428656578064, 91.07142686843872, 100.0, 82.14285969734192, 89.28571343421936, 87.5]
